![Shopping trolley in front of a laptop](./iStock-1249219777.jpg)

It's simple to buy any product with a click and have it delivered to your door. Online shopping has been rapidly evolving over the last few years, making our lives easier. But behind the scenes, e-commerce companies face a complex challenge that needs to be addressed. 

Uncertainty plays a big role in how the supply chains plan and organize their operations to ensure that the products are delivered on time. These uncertainties can lead to challenges such as stockouts, delayed deliveries, and increased operational costs.

You work for the Sales & Operations Planning (S&OP) team at a multinational e-commerce company. They need your help to assist in planning for the upcoming end-of-the-year sales. They want to use your insights to plan for promotional opportunities and manage their inventory. This effort is to ensure they have the right products in stock when needed and ensure their customers are satisfied with the prompt delivery to their doorstep.


## The Data

You are provided with a sales dataset to use. A summary and preview are provided below.

# Online Retail.csv

| Column     | Description              |
|------------|--------------------------|
| `'InvoiceNo'` | A 6-digit number uniquely assigned to each transaction |
| `'StockCode'` | A 5-digit number uniquely assigned to each distinct product |
| `'Description'` | The product name |
| `'Quantity'` | The quantity of each product (item) per transaction |
| `'UnitPrice'` | Product price per unit |
| `'CustomerID'` | A 5-digit number uniquely assigned to each customer |
| `'Country'` | The name of the country where each customer resides |
| `'InvoiceDate'` | The day and time when each transaction was generated `"MM/DD/YYYY"` |
| `'Year'` | The year when each transaction was generated |
| `'Month'` | The month when each transaction was generated |
| `'Week'` | The week when each transaction was generated (`1`-`52`) |
| `'Day'` | The day of the month when each transaction was generated (`1`-`31`) |
| `'DayOfWeek'` | The day of the weeke when each transaction was generated <br>(`0` = Monday, `6` = Sunday) |

Project Instructions

Analyze the Online Retail.csv dataset and build a forecasting model to predict 'Quantity' of products sold.

    Split the data into two sets based on the splitting date, "2011-09-25". All data up to and including this date should be in the training set, while data after this date should be in the test set. Return a pandas DataFrame, pd_daily_train_data, containing, at least, the columns "Country", "StockCode", "InvoiceDate", "Quantity".

    Using your test set, calculate the Mean Absolute Error (MAE) for your forecast model for the 'Quantity' sold? Return a double (float) named mae.

    How many units are expected to be sold during the week 39 of 2011? Store as an integer variable called quantity_sold_w39.

In [51]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.sql.functions import col, dayofmonth, month, year,  to_date, to_timestamp, weekofyear, dayofweek
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Initialize Spark session
my_spark = SparkSession.builder.appName("SalesForecast").getOrCreate()

# Importing sales data
sales_data = my_spark.read.csv(
    "Online Retail.csv", header=True, inferSchema=True, sep=",")

# Convert InvoiceDate to datetime 
sales_data = sales_data.withColumn("InvoiceDate", to_date(
    to_timestamp(col("InvoiceDate"), "d/M/yyyy H:mm")))

In [52]:
sales_data.show()

+---------+---------+--------------------+--------+---------+----------+--------------+-----------+----+-----+----+---+---------+
|InvoiceNo|StockCode|         Description|Quantity|UnitPrice|CustomerID|       Country|InvoiceDate|Year|Month|Week|Day|DayOfWeek|
+---------+---------+--------------------+--------+---------+----------+--------------+-----------+----+-----+----+---+---------+
|   536365|   85123A|WHITE HANGING HEA...|       6|     2.55|     17850|United Kingdom| 2010-01-12|2010|    1|   2| 12|        1|
|   536365|    71053| WHITE METAL LANTERN|       6|     3.39|     17850|United Kingdom| 2010-01-12|2010|    1|   2| 12|        1|
|   536365|   84406B|CREAM CUPID HEART...|       8|     2.75|     17850|United Kingdom| 2010-01-12|2010|    1|   2| 12|        1|
|   536365|   84029G|KNITTED UNION FLA...|       6|     3.39|     17850|United Kingdom| 2010-01-12|2010|    1|   2| 12|        1|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|     3.39|     17850|United Kingdom| 20

In [53]:
sales_data.printSchema()

root
 |-- InvoiceNo: integer (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- InvoiceDate: date (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Week: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)



In [54]:
sales_data.summary().show()

+-------+------------------+------------------+--------------------+-----------------+-----------------+------------------+-----------+------------------+-----------------+------------------+-----------------+------------------+
|summary|         InvoiceNo|         StockCode|         Description|         Quantity|        UnitPrice|        CustomerID|    Country|              Year|            Month|              Week|              Day|         DayOfWeek|
+-------+------------------+------------------+--------------------+-----------------+-----------------+------------------+-----------+------------------+-----------------+------------------+-----------------+------------------+
|  count|            384721|            384721|              384721|           384721|           384721|            384721|     384721|            384721|           384721|            384721|           384721|            384721|
|   mean| 560650.5626960837|27337.415763815174|                null|8.30641165935834

In [55]:
sales_data.describe().show()

+-------+------------------+------------------+--------------------+-----------------+-----------------+------------------+-----------+------------------+-----------------+------------------+-----------------+------------------+
|summary|         InvoiceNo|         StockCode|         Description|         Quantity|        UnitPrice|        CustomerID|    Country|              Year|            Month|              Week|              Day|         DayOfWeek|
+-------+------------------+------------------+--------------------+-----------------+-----------------+------------------+-----------+------------------+-----------------+------------------+-----------------+------------------+
|  count|            384721|            384721|              384721|           384721|           384721|            384721|     384721|            384721|           384721|            384721|           384721|            384721|
|   mean| 560650.5626960837|27337.415763815174|                null|8.30641165935834

In [56]:
from pyspark.sql import functions as F

sales_data = sales_data.withColumn("Year", F.year("InvoiceDate")) \
                        .withColumn("Month", F.month("InvoiceDate")) \
                        .withColumn("Day", F.dayofmonth("InvoiceDate")) \
                        .withColumn("Week", F.weekofyear("InvoiceDate")) \
                        .withColumn("DayOfWeek", F.date_format("InvoiceDate", "E"))

daily_sales = sales_data.groupBy(
    "Country", "StockCode", "InvoiceDate", "Year", "Month", "Day", "Week", "DayOfWeek"
).agg(
    {"Quantity": "sum", "UnitPrice": "avg"}
)
daily_sales = daily_sales.withColumnRenamed("sum(Quantity)", "Quantity") \
                            .withColumnRenamed("avg(UnitPrice)", "AvgUnitPrice")
daily_sales.show(10)

+--------------+---------+-----------+----+-----+---+----+---------+------------+--------+
|       Country|StockCode|InvoiceDate|Year|Month|Day|Week|DayOfWeek|AvgUnitPrice|Quantity|
+--------------+---------+-----------+----+-----+---+----+---------+------------+--------+
|United Kingdom|    22180| 2010-01-12|2010|    1| 12|   2|      Tue|        9.95|       8|
|United Kingdom|    84347| 2010-01-12|2010|    1| 12|   2|      Tue|        2.55|      23|
|United Kingdom|    22785| 2010-01-12|2010|    1| 12|   2|      Tue|        6.75|       2|
|        Norway|    85150| 2010-01-12|2010|    1| 12|   2|      Tue|        2.55|      12|
|          EIRE|    22150| 2010-01-12|2010|    1| 12|   2|      Tue|        1.95|      12|
|United Kingdom|    37448| 2010-01-12|2010|    1| 12|   2|      Tue|        1.49|      12|
|United Kingdom|    85216| 2010-02-12|2010|    2| 12|   6|      Fri|        0.65|      12|
|United Kingdom|    22737| 2010-02-12|2010|    2| 12|   6|      Fri|        1.65|      30|

In [57]:
split_date = "2011-09-25"

train_data = daily_sales.filter(F.col("InvoiceDate") <= F.lit(split_date))
test_data = daily_sales.filter(F.col("InvoiceDate") > F.lit(split_date))

pd_daily_train_data = train_data.toPandas()

In [58]:
print("Train data max date:", train_data.agg(F.max("InvoiceDate")).collect()[0][0])
print("Test data min date:", test_data.agg(F.min("InvoiceDate")).collect()[0][0])


Train data max date: 2011-09-25
Test data min date: 2011-09-26


In [59]:
country_indexer = StringIndexer(
    inputCol="Country",
    outputCol="CountryIndex"
).setHandleInvalid("keep")

stock_code_indexer = StringIndexer(
    inputCol="StockCode",
    outputCol="StockCodeIndex"
).setHandleInvalid("keep")


In [60]:
assembler = VectorAssembler(
    inputCols=["CountryIndex", "StockCodeIndex", "Year", "Month", "Day", "Week", "AvgUnitPrice"],
    outputCol="features"
)


In [61]:
rf = RandomForestRegressor(
    featuresCol="features",
    labelCol="Quantity",
    maxBins=4000
)

pipeline = Pipeline(stages=[country_indexer, stock_code_indexer, assembler, rf])
model = pipeline.fit(train_data)


25/10/19 05:48:44 WARN DAGScheduler: Broadcasting large task binary with size 1612.8 KiB


25/10/19 05:48:45 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB


In [62]:
test_predictions = model.transform(test_data)
test_predictions = test_predictions.withColumn("prediction", col("prediction").cast("double"))


In [63]:
mae_evaluator = RegressionEvaluator(
    labelCol="Quantity",
    predictionCol="prediction",
    metricName="mae"
)
mae = mae_evaluator.evaluate(test_predictions)

print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 9.150063430518875


In [64]:
weekly_sales = test_predictions.groupBy("Year", "Week").agg(
    F.sum("prediction").alias("TotalPredictedQuantity")
)

week_39_sales = weekly_sales.filter((F.col("Year") == 2011) & (F.col("Week") == 39))
quantity_sold_w39 = int(week_39_sales.select("TotalPredictedQuantity").collect()[0][0])
print("Predicted quantity sold in Week 39 of 2011:", quantity_sold_w39)

Predicted quantity sold in Week 39 of 2011: 87501
